In [2]:
from score import featurize, test

ModuleNotFoundError: No module named 'rdkit'

## Basic Example

In [2]:
import numpy as np
import pandas as pd
import mordred.AtomCount

In [6]:
features = [
    mordred.AtomCount.AtomCount('C'),
    mordred.AtomCount.AtomCount('H'),
    mordred.AtomCount.AtomCount('O'),
    mordred.AtomCount.AtomCount('N'),

#     mordred.CPSA.FPSA(5),
#     mordred.CPSA.PPSA(5),
#     mordred.CPSA.FNSA(5),
#     mordred.CPSA.PNSA(5),
#     mordred.CPSA.DPSA(5),   # Would like to see if difference in charged partial surface area will cover the previous 4 features
    
    mordred.CarbonTypes.CarbonTypes(1,2),
    mordred.HydrogenBond.HBondAcceptor(),
    mordred.HydrogenBond.HBondDonor(),
    mordred.TopoPSA.TopoPSA(False)
]

In [12]:
X_train, y_train = featurize('Data/Solubility/dataset-F.csv',features)

100%|██████████| 1210/1210 [00:02<00:00, 428.61it/s]


In [13]:
from sklearn.linear_model import LinearRegression

In [14]:
model = LinearRegression()

In [15]:
model.fit(X_train,y_train)

LinearRegression()

In [16]:
test(features,model)

100%|██████████| 1291/1291 [00:02<00:00, 444.16it/s]


1.166639528565129

# Team Electron
We have decided to include our comments on this notebook as sort of a "storytelling" presentation. Feel free to read our comments as you go!

## Background

This section begins with some of the analysis I found from homework three. First of all, by simply trial and error of many different models (NO FREE LUNCH), I found that the validation error was able to be generalized to 0.6 for the Decision Tree Regressor model. This does not mean we will choose this model in our final submission, but we will begin testing there and looking at similar alternatives.

One of the observations I noted was that the difference between the validation and training errors were not yet converging. That is, the difference was still consistently decreasing even at 100% of the dataset. At the same time, there were no noticeable jitters/extra noise around either the training or the validation datasets, which indicates that the model has the potential to be even more accurate.

The primary limiting factor then (ignoring parameter optimization) is the amount of data. I would hypothesize that with more data, we could optimize this model. Luckily for us, Prof. Tristan gave us multiple datasets to work with. Thus, step one will be preparing our data by combining our datasets. This way, we can take advantage of as much data as possible. Analytically, this only has benefits (will do no harm) as determined in the Hoeffding Inequality.

Let's get started! :)
-RJ

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import validation_curve
from sklearn.model_selection import learning_curve
from sklearn.model_selection import cross_val_score

from sklearn.linear_model import Ridge
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import GridSearchCV

# Copy over some useful functions from HW 3
def get_pipeline(models, degree):
    model_tups = [('model-0', PolynomialFeatures(degree))]
    for i in range(len(models)):
        model_tups.append(('model-'+str(i+1), models[i]))
    return Pipeline(model_tups)



## Step 1. Data Preparation

In [29]:
import pandas as pd

# Combine Datasets
def generate_combined_dataset(dir_path, fnames, out_name="", debug=False):
    combined = None
    for f in fnames:
        df = pd.read_csv(dir_path + f)
        if combined is None:
            combined = df.copy()
        else:
            ## TODO: Check that 
            combined = pd.concat([combined, df])
        if debug:
            print("Added " + str(f) + " with dim", df.shape)
    if debug:
        print("Our combined dataset has", combined.shape[0], "rows with", combined.shape[1], "features.")
    return combined
    
# Restriction: "It is forbidden to use any dataset other than A, B, C, D, F, G, H, I"
# So, we make sure we only use these datasets in our data preparation
generate_combined_dataset("../../Data/Solubility/",
                          ["dataset-A.csv", "dataset-B.csv", "dataset-C.csv",
                          "dataset-D.csv", "dataset-F.csv", "dataset-G.csv",
                          "dataset-H.csv", "dataset-I.csv"])

Added dataset-A.csv with dim (6110, 6)
Added dataset-B.csv with dim (4650, 6)
Added dataset-C.csv with dim (2603, 6)
Added dataset-D.csv with dim (2115, 6)
Added dataset-F.csv with dim (1210, 6)
Added dataset-G.csv with dim (1144, 6)
Added dataset-H.csv with dim (578, 6)
Added dataset-I.csv with dim (94, 6)
Our combined dataset has 18504 rows with 6 features.


### Step 2. Model Selection

In [ ]:
from sklearn.tree import DecisionTreeRegressor
print("----- DECISION TREE REGRESSOR -----")
# Needs more data (would combine datasets to achieve this) or should reduce complexity
models = [DecisionTreeRegressor()]
validation_curve_poly_degree(X_train, y_train, X_test, y_test, models=models)
learning_curves_data_used(X_train, y_train, X_test, y_test, models=models)

pl = Pipeline([('poly', PolynomialFeatures()), ('pca', PCA()), ('ridge', Ridge())])

# Center training X
scaler = StandardScaler()
scaler.fit(X_train)
X_train_cent = scaler.transform(X_train)

# Center testing X
scaler = StandardScaler()
scaler.fit(X_test)
X_test_cent = scaler.transform(X_test)

parameters = {'poly__degree': [1,2,3], 'pca__n_components': [2,3,4,5,9]}
gscv = GridSearchCV(pl, parameters)
gscv.fit(X_train_cent, y_train)
# print(gscv.cv_results_)
# gscv is best model

def print_best_model(grid_search_model):
    ranks = grid_search_model.cv_results_["rank_test_score"]
    best_index = np.where(ranks==1)[0][0]
    print("The best model had parameters:", grid_search_model.cv_results_["params"][best_index])

print_best_model(gscv)
mean_absolute_error(y_test, gscv.predict(X_test_cent))